Float Version

In [15]:
import numpy as np

In [16]:
A = np.array([[3, 3],
              [2, 5],
              [0 ,6]])
B_T = np.array([[3, 2, 3],
                [2, 6, 1]])

In [17]:
def pivoting(tableau, basic_vars, pivot):
    
    ind_nonpositive = tableau[:, pivot] <= 0
    with np.errstate(divide='ignore', invalid='ignore'):
        ratios = tableau[:, -1] / tableau[:, pivot]
    ratios[ind_nonpositive] = np.inf
            
    row_min = ratios.argmin()
    tableau[row_min, :] /= tableau[row_min, pivot]
    
    for i in range(tableau.shape[0]):
        if i != row_min:
            tableau[i, :] -= tableau[i, pivot] * tableau[row_min, :]
    basic_vars[row_min], pivot = pivot, basic_vars[row_min]
    
    return pivot

In [18]:
def create_tableau(A, B):
    B_T = B.T
    m, n = A.shape
    tableaus = []
    
    for i in range(2):
        tableaus.append(np.empty((A.shape[1-i], m+n+1)))
        tableaus[i][:, :m] = [B_T, np.identity(m)][i]
        tableaus[i][:, m:m+n] = [np.identity(n), A][i]
        tableaus[i][:, -1] = 1
        
    basic_vars_list = [np.arange(m, m+n), np.arange(m)]
    
    return tableaus, basic_vars_list

In [45]:
def Lemke_Howson(tableaus, basic_vars_list, init_pivot, return_tableau = False, show_results = False):
    
    m, n = tableaus[1].shape[0], tableaus[0].shape[0]
    pivot = init_pivot
    init_player = int((basic_vars_list[0]==init_pivot).any())
    players = [init_player, 1 - init_player]
    
    while True:
        for i in players:
            pivot = pivoting(tableaus[i], basic_vars_list[i], pivot)
            if show_results:
                print(tableaus[i], basic_vars_list[i])
            if pivot == init_pivot:
                break
        else:
            continue
        break
        
    #summarize the found NE
    normalized = np.empty(m+n)
    out = np.zeros(m+n)
    for i, (start, num) in enumerate(zip([0, m], [m, n])):
        ind = basic_vars_list[i] < start + num if i == 0 else start <= basic_vars_list[i]
        out[basic_vars_list[i][ind]] = tableaus[i][ind, -1]
        s = out[start:start+num].sum()
        if s != 0:
            for j in range(start,start+num):
                normalized[j] = out[j] / s
        else:
            normalized[start:start+num] = np.zeros(num)
    actions = normalized[:m], normalized[m:]
    
    if show_results:
        print(out[:m], out[m:])
        
    if return_tableau:
        return actions, tableaus, basic_vars_list
    
    return actions

In [33]:
tableaus, basic_vars_list = create_tableau(A, B_T.T)
Lemke_Howson(tableaus, basic_vars_list, 1, return_tableau = True)

((array([ 0.        ,  0.33333333,  0.66666667]),
  array([ 0.33333333,  0.66666667])),
 [array([[ 0.875 ,  0.    ,  1.    ,  0.375 , -0.125 ,  0.25  ],
         [ 0.1875,  1.    ,  0.    , -0.0625,  0.1875,  0.125 ]]),
  array([[ 1.        , -1.5       ,  0.75      ,  0.        ,  0.        ,
           0.25      ],
         [ 0.        ,  0.5       , -0.41666667,  1.        ,  0.        ,
           0.08333333],
         [ 0.        ,  0.        ,  0.16666667,  0.        ,  1.        ,
           0.16666667]])],
 [array([2, 1]), array([0, 3, 4])])

In [21]:
#Enumerate all NEs that can be found by Lemke Howson algorithm
def Lemke_Howson_all(A, B):
    
    #define the k of V_k
    k = 0
    
    tableaus, basic_vars = create_tableau(A, B)
    m, n = A.shape
    NEs = []
    basic_vars_found = []
    player = (m <= n)
    init_pivot = k
    
    action, tableau, basic_vars = Lemke_Howson(tableaus, basic_vars, init_pivot, return_tableau = True)
    NEs.append(action)
    basic_vars_found.append(np.sort(basic_vars[player]))
    
    for a in range(m+n):
        if a == k:
            continue
        tableaus, basic_vars = create_tableau(A, B)
        init_pivot = a
        action, tableau, basic_vars = Lemke_Howson(tableaus, basic_vars, init_pivot, return_tableau = True)
        for arr in basic_vars_found:
            if np.array_equal(np.sort(basic_vars[player]), arr):
                break
        else:
            NEs.append(action)
            basic_vars_found.append(np.sort(basic_vars[player]))
            
            #start from the found NE by dropping k
            action, tableau, basic_vars = Lemke_Howson(tableaus, basic_vars, k, return_tableau = True)
            NEs.append(action)
            basic_vars_found.append(np.sort(basic_vars[player]))
    return NEs

In [22]:
Lemke_Howson_all(A, B_T.T)

[(array([ 1.,  0.,  0.]), array([ 1.,  0.])),
 (array([ 0.        ,  0.33333333,  0.66666667]),
  array([ 0.33333333,  0.66666667])),
 (array([ 0.8,  0.2,  0. ]), array([ 0.66666667,  0.33333333]))]

Integer Version

In [23]:
import sympy as sp

In [24]:
def create_tableau_int(A, B):
    B_T = B.T
    m, n = A.shape
    tableaus = []
    
    for i in range(2):
        tableaus.append(np.empty((A.shape[1-i], m+n+1), dtype=int))
        tableaus[i][:, :m] = [B_T, np.identity(m)][i]
        tableaus[i][:, m:m+n] = [np.identity(n), A][i]
        tableaus[i][:, -1] = 1
        
    basic_vars_list = [np.arange(m, m+n), np.arange(m)]
    
    return tableaus, basic_vars_list

In [25]:
def pivoting_int(tableau, basic_vars, pivot, previous_pivot):
    
    with np.errstate(divide='ignore'):
        ratios = tableau[:, -1] / tableau[:, pivot]
    for i in range(len(ratios)):
        if ratios[i] < 0:
            ratios[i] = np.inf
    row_min = ratios.argmin()
    element = tableau[row_min, pivot]
    
    for i in range(tableau.shape[0]):
        if i != row_min:
            elementi = tableau[i, pivot]
            tableau[i, :] *= element
            tableau[i, :] -= elementi * tableau[row_min, :]
            tableau[i, :] //= previous_pivot
    basic_vars[row_min], pivot = pivot, basic_vars[row_min]
    
    return pivot, element

In [44]:
def Lemke_Howson_int(tableaus, basic_vars_list, init_pivot, return_tableau = False, show_results = False):
    
    m, n = tableaus[1].shape[0], tableaus[0].shape[0]
    pivot = init_pivot
    init_player = int((basic_vars_list[0]==init_pivot).any())
    players = [init_player, 1 - init_player]
    
    #store the pivots of previous turn
    pivots_element = [1, 1]
    
    while True:
        for i in players:
            pivot, pivots_element[i] = pivoting_int(tableaus[i], basic_vars_list[i], pivot, pivots_element[i])
            if show_results:
                print(tableaus[i], basic_vars_list[i])
            if pivot == init_pivot:
                break
        else:
            continue
        break
        
    #summarize the found NE
    normalized = np.empty(m+n, np.object_)
    out = np.zeros(m+n)
    for i, (start, num) in enumerate(zip([0, m], [m, n])):
        ind = basic_vars_list[i] < start + num if i == 0 else start <= basic_vars_list[i]
        out[basic_vars_list[i][ind]] = tableaus[i][ind, -1]
        s = out[start:start+num].sum()
        if s != 0:
            for j in range(start,start+num):
                normalized[j] = sp.Rational(sp.S(out[j]), sp.S(s))
        else:
            normalized[start:start+num] = sp.Rational(0)
    actions = normalized[:m], normalized[m:]
    
    if show_results:
        print(normalized[:m], normalized[m:])
        
    if return_tableau:
        return actions, tableaus, basic_vars_list
    
    return actions

In [27]:
Lemke_Howson_int(*create_tableau(A, B_T.T), 1, show_results = True)

[[ 14.  -0.  16.   6.  -2.   4.]
 [  2.   6.   1.   0.   1.   1.]] [3 1]
[[  6.  -0.  -3.  18.  -0.   3.]
 [ -0.   6.  -5.  12.  -0.   1.]
 [  0.   0.   1.   0.   6.   1.]] [0 1 4]
[[ 14.  -0.  16.   6.  -2.   4.]
 [  3.  16.  -0.  -1.   3.   2.]] [2 1]
[[ 12. -18.   9.  -0.  -0.   3.]
 [ -0.   6.  -5.  12.  -0.   1.]
 [ -0.  -0.   2.  -0.  12.   2.]] [0 3 4]
[0 1/3 2/3] [1/3 2/3]


(array([0, 1/3, 2/3], dtype=object), array([1/3, 2/3], dtype=object))

In [28]:
#Enumerate all NEs that can be found by Lemke Howson algorithm
def Lemke_Howson_all_int(A, B):
    
    #define the k of V_k
    k = 0
    
    tableaus, basic_vars = create_tableau_int(A, B)
    m, n = A.shape
    NEs = []
    basic_vars_found = []
    player = (m <= n)
    init_pivot = k
    
    action, tableau, basic_vars = Lemke_Howson_int(tableaus, basic_vars, init_pivot, return_tableau = True)
    NEs.append(action)
    basic_vars_found.append(np.sort(basic_vars[player]))
    
    for a in range(m+n):
        if a == k:
            continue
        tableaus, basic_vars = create_tableau_int(A, B)
        init_pivot = a
        action, tableau, basic_vars = Lemke_Howson_int(tableaus, basic_vars, init_pivot, return_tableau = True)
        for arr in basic_vars_found:
            if np.array_equal(np.sort(basic_vars[player]), arr):
                break
        else:
            NEs.append(action)
            basic_vars_found.append(np.sort(basic_vars[player]))
            
            #start from the found NE by dropping k
            action, tableau, basic_vars = Lemke_Howson_int(tableaus, basic_vars, k, return_tableau = True)
            NEs.append(action)
            basic_vars_found.append(np.sort(basic_vars[player]))
    return NEs

In [29]:
Lemke_Howson_all_int(A, B_T.T)

[(array([1, 0, 0], dtype=object), array([1, 0], dtype=object)),
 (array([0, 1/3, 2/3], dtype=object), array([1/3, 2/3], dtype=object)),
 (array([1/5, 4/5, 0], dtype=object), array([2/3, 1/3], dtype=object))]

In [36]:
C = np.array([[3, 3],
              [2, 5],
              [0 ,6]])
D_T = np.array([[3, 2, 3],
                [3, 6, 1]])

In [46]:
Lemke_Howson_all(C, D_T.T)

[(array([ 1.,  0.,  0.]), array([ 1.,  0.])),
 (array([ 0.        ,  0.33333333,  0.66666667]),
  array([ 0.33333333,  0.66666667])),
 (array([ 0.,  0.,  0.]), array([ 0.,  0.]))]